In [1]:
import csv
import os
from os.path import dirname, abspath
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

data_dir = os.path.abspath(os.path.curdir) + "/data/minute"
available_files = os.listdir(data_dir)
available_symbols = []
current_symbols = []
data = {}

for fileName in available_files:
    if fileName.endswith("usd.csv"):
        name = fileName[0:-7]
        available_symbols.append(name)

symbols = widgets.Dropdown(
    options=available_symbols,
    description='Symbol:',
    disabled=False
)

display(symbols)

add_symbol_button = widgets.Button(description="Add Symbol")
remove_symbol_button = widgets.Button(description="Remove Symbol")

output = widgets.Output()


def add_symbol(b):
    current_symbols.append(symbols.value)
    data[symbols.value] = fetch_data(symbols.value)
    weights_equal = get_weights_equal()
    with output:
        print("Added "+ symbols.value, 'Current Assets: '+ ", ".join(current_symbols))

def remove_symbol(b):
    current_symbols.remove(symbols.value)
    del data[symbols.value]
    weights_equal = get_weights_equal()
    with output:
        print("Removed "+ symbols.value, 'Current Assets: '+ ", ".join(current_symbols))

add_symbol_button.on_click(add_symbol)
remove_symbol_button.on_click(remove_symbol)

display(add_symbol_button, remove_symbol_button, output)

def fetch_data(symbol):
    file = pd.read_csv(data_dir+"/"+symbol+"usd.csv")
    file['time'] = pd.to_datetime(file["time"], unit="ms")
    file.set_index('time',drop=False, inplace=True)
    if "Unnamed: 0" in file.columns:
        del file["Unnamed: 0"]
    return file

def get_weights_equal():
    weights = {}
    for symbol in current_symbols:
        weights[symbol] = 1/len(current_symbols)
    return weights

def merge_timeseries(datasets, label):
    dfs = []
    for asset, df in datasets.items():
        dfs.append(df[label].rename(asset))
    merged = pd.concat(dfs, axis=1)
    merged.ffill(inplace=True)
    merged.fillna(0, inplace=True)
    return merged

# import pandas_gbq

# def gbq_store(datasets, table, project_id="shed-enterprises-310921"):
#     for dataset in datasets:
#             pandas_gbq.to_gbq(datasets[dataset],dataset + "." + table, project_id=project_id, if_exists="replace", table_schema = [{'name':'time','type': 'TIMESTAMP'}])

FileNotFoundError: [Errno 2] No such file or directory: '/private/var/mobile/Containers/Data/Application/CB699D45-C2C0-493F-87D3-B2A359F50B76/tmp/NSIRD_Carnets_EZjXR9/data/minute'

Metrics Functions



In [ ]:
import numpy as np
current_frequencies = []

def change_percent(frequency, dataset, append_column=True):
    # Get price value for each offset
    value = dataset["close"].pct_change(freq=frequency)
    if append_column:
        dataset[frequency+"_percent_change"] = value
    return value

def scaled_close(dataset, append_column=True):
    # Get price value for each offset
    value = dataset["close"] / max(dataset["close"])
    if append_column:
        dataset["scaled_close"] = value
    return value

def change_log(frequency, dataset, append_column=True):
    # Get price value for each offset
    if frequency+"_percent_change" in dataset:
        value = dataset[frequency+"_percent_change"].apply(lambda x: np.log(1+x))
    else:
        value = change_percent(frequency, dataset, False).apply(lambda x: np.log(1+x))
    if append_column:
        dataset[frequency+"_log_change"] = value 
    return value

def change_usd(frequency, dataset, append_column=True):
    # Get price value for each offset
    if frequency+"_percent_change" in dataset:
        value = dataset.close * dataset[frequency+"_percent_change"]
    else:
        value = dataset.close * change_percent(frequency, dataset, False)
    if append_column:
        dataset[frequency+"_usd_change"] = value 
    return dataset

def asset_statistics(datasets, frequency):
    stats = {}
    log_change = merge_timeseries(datasets, frequency + "_log_change")
    stats["var"] = log_change.var()
    stats["cov"] = log_change.cov()
    stats["corr"] = log_change.corr()
    return stats

from IPython.display import display
import ipywidgets as widgets

output = widgets.Output()

available_frequencies = {
'B':'business day frequency',
'C':'custom business day frequency',
'D':'calendar day frequency',
'W':'weekly frequency',
'M':'month end frequency',
'SM':'semi-month end frequency (15th and end of month)',
'BM':'business month end frequency',
'CBM':'custom business month end frequency',
'MS':'month start frequency',
'SMS':'semi-month start frequency (1st and 15th)',
'BMS':'business month start frequency',
'CBMS':'custom business month start frequency',
'Q':'quarter end frequency',
'BQ':'business quarter end frequency',
'QS':'quarter start frequency',
'BQS':'business quarter start frequency',
'A, Y ':'year end frequency',
'BA, BY':'business year end frequency',
'AS, YS':'year start frequency',
'BAS, BYS':'business year start frequency',
'BH':'business hour frequency',
'H':'hourly frequency',
'min':'minutely frequency',
'S':'secondly frequency',
'L, ms':'milliseconds',
'U, us':'microseconds',
'N':'nanoseconds',
}

available_frequencies_dropdown = widgets.Dropdown(
    options= dict((v,k) for k,v in available_frequencies.items()),
    description='Frequency:',
    disabled=False
)

interval_dropdown = widgets.IntSlider(
    value=1,
    min=1,
    max=100,
    step=1,
    description='Interval:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


calculate_metrics_button = widgets.Button(description="Calculate Metrics")

def run_asset_metrics(b):
    for frequency in current_frequencies:
        for dataset in data:
            change_percent(frequency, data[dataset])
            change_log(frequency, data[dataset])
            change_usd(frequency, data[dataset])
            scaled_close(data[dataset])
    with output:
        print(asset_statistics(data, frequency))
calculate_metrics_button.on_click(run_asset_metrics)


add_frequency_button = widgets.Button(description="Add Frequency")

def add_frequency(b):
    current_frequencies.append(str(interval_dropdown.value) + available_frequencies_dropdown.value)
    with output:
        print("Added "+ str(interval_dropdown.value) + available_frequencies_dropdown.value, 'Current Frequencies: '+ ", ".join(current_frequencies))

add_frequency_button.on_click(add_frequency)


remove_frequency_button = widgets.Button(description="Remove Frequency")

def remove_frequency(b):
    current_frequencies.remove(str(interval_dropdown.value) + available_frequencies_dropdown.value)
    del data[symbols.value]
    weights_equal = get_weights_equal()
    with output:
        print("Removed "+ str(interval_dropdown.value) + available_frequencies_dropdown.value, 'Current Frequencies: '+ ", ".join(current_frequencies))

remove_frequency_button.on_click(remove_frequency)

display(available_frequencies_dropdown, interval_dropdown,add_frequency_button, remove_frequency_button, calculate_metrics_button, output)


In [ ]:
from plotly.offline import iplot
import plotly
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'
# plotly.offline.init_notebook_mode()

fig = go.Figure()

# This is a scaled close graph, which can be used to compare asset returns on scaled close. Scaled close is the close / max close, meaning everything is maxed out at 1
merged = merge_timeseries(data, "scaled_close")
for symbol in current_symbols:
    fig.add_trace(
        go.Scatter(
            x = merged.index,
            y = merged[symbol],
            name = symbol
        )
    )
fig.update_layout(
    title = "Scaled Closing Price of " + ", ".join(current_symbols),
    xaxis_title = 'Date',
    yaxis_title = f'Scaled Close',
)
# fig.update_yaxes(tickprefix='$')
fig.show()


# for frequency in current_frequencies:
#     for idx, name in enumerate(data):
#         fig = fig.add_trace(
#             go.Scatter(
#                 x = data[name].index,
#                 y = data[name].colu,
#                 name = name,
#             )
#         )
    # for dataset in data:
    #     merged = merge_timeseries(data, frequency + "_percent_change")
    #     filtered = merged['2016-01-01':'2021-01-01']


# for idx, name in enumerate(data):
#     fig = fig.add_trace(
#         go.Scatter(
#             x = data[name].index,
#             y = data[name].close,
#             name = name,
#         )
#     )

